In [ ]:
//---- imports for HIPO4 library
import org.jlab.jnp.hipo4.io.*;
import org.jlab.jnp.hipo4.data.*;
//---- imports for GROOT library
import org.jlab.groot.data.*;
import org.jlab.groot.graphics.*;
//---- imports for PHYSICS library
import org.jlab.jnp.physics.*;
import org.jlab.jnp.reader.*;


HipoReader reader = new HipoReader(); // Create a reader obejct
reader.open("../../data/out_6489_2xx_3xx.hipo"); // open a file

In [ ]:
EmbeddedCanvas ec = new EmbeddedCanvas(800,800);

In [ ]:
Event     event = new Event();
Bank  particles = new Bank(reader.getSchemaFactory().getSchema("REC::Particle"));

In [ ]:
EventFilter  nDVCSFilter = new EventFilter("11:2112:22:Xn:X-");
EventFilter  pDVCSFilter = new EventFilter("11:2212:22:Xn:X-");

In [ ]:
reader.getEvent(event,0); //Reads the first event and resets to the begining of the file
int times=0;
int pid;
LorentzVector  vBeam   = new LorentzVector(0.0,0.0,10.6,10.6);
LorentzVector  vTarget = new LorentzVector(0.0,0.0,0.0,0.938);
LorentzVector  electron = new LorentzVector();
Particle  proton = new Particle();
Particle neutron = new Particle();
Particle deuteron = new Particle();
LorentzVector       vpW = new LorentzVector();
LorentzVector      vpQ2 = new LorentzVector();
double pXbj;
LorentzVector       vnW = new LorentzVector();
LorentzVector      vnQ2 = new LorentzVector();
double nXbj;

H1F  pW = new H1F("pW" ,100, 0.5, 4.0);
H1F pQ2 = new H1F("pQ2",100, 0.1, 4.0);
pW.setTitleX("W [GeV]");
pQ2.setTitleX("Q^2 [GeV/c^2]");
H1F  nW = new H1F("nW" ,100, 0.5, 4.0);
H1F nQ2 = new H1F("nQ2",100, 0.1, 4.0);
nW.setTitleX("W [GeV]");
nQ2.setTitleX("Q^2 [GeV/c^2]");
H2F nWvsnQ2 = new H2F("nDVCS W vs Q2", "nDVCS W vs Q2", 100,0.1,5,100,0.1,10);
H2F pWvsnQ2 = new H2F("pDVCS W vs Q2", "pDVCS W vs Q2", 100,0.1,5,100,0.1,10);
H2F pThvsPhi = new H2F("proton Theta vs Phi","proton Theta vs Phi",100,0.1,3,100,0.1,3.5);
H2F nThvsPhi = new H2F("neutron Theta vs Phi","neutron Theta vs Phi",100,0.1,3,100,0.1,3.5);
H2F dThvsPhi = new H2F("deuteron Theta vs Phi","deuteron Theta vs Phi",100,0.1,3,100,0.1,3);
H2F nQ2vsXbj = new H2F("neutron Q2 vs Xbj","neutron Q2 vs Xbj",100,0.1,10,100,0.1,1);
H2F pQ2vsXbj = new H2F("proton Q2 vs Xbj","protonQ2 vs Xbj",100,0.1,10,100,0.1,1);

while(reader.hasNext()==true){

reader.nextEvent(event);
event.read(particles);
    
PhysicsEvent physEvent = DataManager.getPhysicsEvent(10.6,particles);
    
neutron = physEvent.getParticleByPid(2112,0);
nThvsPhi.fill(neutron.phi(),neutron.theta());
    
proton = physEvent.getParticleByPid(2212,0);
pThvsPhi.fill(proton.phi(),proton.theta());

deuteron = physEvent.getParticleByPid(45,0);
dThvsPhi.fill(deuteron.phi(),deuteron.theta());

if(particles.getRows()>0){
    pid = particles.getInt("pid",0);
    
    if(nDVCSFilter.isValid(physEvent)==true){
        if(pid==11){
           electron.setPxPyPzM(
               particles.getFloat("px",0), 
               particles.getFloat("py",0),
               particles.getFloat("pz",0),
               0.0005
               );
            
        vnW.copy(vBeam);
        vnW.add(vTarget).sub(electron);
        
        vnQ2.copy(vBeam);
        vnQ2.sub(electron);
            
        nW.fill(vnW.mass());
        nQ2.fill(-vnQ2.mass2());
            
        nWvsnQ2.fill(vnW.mass(),-vnQ2.mass2());
        
        nXbj = (-vnQ2.mass2())/(2*proton.mass()*(vBeam.p()-electron.p()));
        nQ2vsXbj.fill(-vnQ2.mass2(),nXbj);
        }
    }
    
    if(pDVCSFilter.isValid(physEvent)==true){
       if(pid==11){
           electron.setPxPyPzM(
               particles.getFloat("px",0), 
               particles.getFloat("py",0),
               particles.getFloat("pz",0),
               0.0005
               );
        vpW.copy(vBeam);
        vpW.add(vTarget).sub(electron);
        
        vpQ2.copy(vBeam);
        vpQ2.sub(electron);
            
        pW.fill(vpW.mass());
        pQ2.fill(-vpQ2.mass2());  
        
        pWvsnQ2.fill(vpW.mass(),-vpQ2.mass2());
           
        pXbj = (-vpQ2.mass2())/(2*proton.mass()*(vBeam.p()-electron.p()));
        pQ2vsXbj.fill(-vpQ2.mass2(),pXbj);
            }
        }
    }
//times++;
//System.out.println("loop has looped " + times + "times");
}

ec.divide(2,2);
ec.cd(0).draw(dThvsPhi);
ec.cd(1).draw(nQ2vsXbj);
ec.cd(2).draw(nThvsPhi);
ec.cd(3).draw(pThvsPhi);


ec.getScreenShot();